In [6]:
!pip install torch
!pip install -q -U accelerate peft bitsandbytes transformers trl einops

In [7]:
import os
import torch
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
)
from trl import SFTTrainer

In [8]:
def select_samples(dataset, total_samples, step):
    selected_indices = list(range(0, total_samples, step))
    return dataset.select(selected_indices)

In [9]:
snli_dataset = load_dataset("snli")

train_dataset = select_samples(snli_dataset['train'], total_samples=550000, step=550)
test_dataset = select_samples(snli_dataset['test'], total_samples=10000, step=100)
val_dataset = select_samples(snli_dataset['validation'], total_samples=10000, step=100)

train_dataset = train_dataset.map(lambda x: {'text': f"PREMISE: {x['premise']}  HYPOTHESIS: {x['hypothesis']}  PREDICT THE LABEL (0 for entailment, 1 for neutral, 2 for contradiction):"})
test_dataset = test_dataset.map(lambda x: { 'text': f"PREMISE: {x['premise']}  HYPOTHESIS: {x['hypothesis']}  PREDICT THE LABEL (0 for entailment, 1 for neutral, 2 for contradiction):"})
val_dataset = val_dataset.map(lambda x: {'text': f"PREMISE: {x['premise']}  HYPOTHESIS: {x['hypothesis']}  PREDICT THE LABEL (0 for entailment, 1 for neutral, 2 for contradiction):"})

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [5]:
base_model = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map={"": 0}
)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

In [7]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 *  trainable_params / all_param}")

In [8]:
peft_config = LoraConfig(
    r=32,                   # default=8, higher value for more representational capacity
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="TEXT_CLASSIFICATION_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
        "dense"
    ]
)

model = PeftModel(model, peft_config)
print_trainable_parameters(model)

trainable params: 22691840 || all params: 1544084480 || trainable%: 1.4695983473650354


In [32]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    fp16=False,
    bf16=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    learning_rate=2e-5,
    weight_decay=0.001,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=25,
    evaluation_strategy="steps",  # Evaluate during training at each eval_step
    eval_steps=25                 # Frequency of evaluation
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Include validation dataset for evaluations during training
    peft_config=peft_config,
    dataset_text_field="text",  # Use appropriate fields for NLI tasks ('premise', 'hypothesis', 'label')
    tokenizer=tokenizer,
    max_seq_length= None,
    args=training_arguments,
)

trainer.can_return_loss=True

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.war

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [34]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
25,3.022900,2.985771
50,3.006900,2.867159
75,2.772800,2.573908
100,2.455200,2.276181
125,2.244600,2.108260
150,2.112900,2.064806
175,2.168600,2.042719
200,1.940300,1.987384
225,2.118800,2.031471
250,1.989700,2.039788


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=1250, training_loss=1.958230551147461, metrics={'train_runtime': 3964.1522, 'train_samples_per_second': 1.261, 'train_steps_per_second': 0.315, 'total_flos': 4556902151208960.0, 'train_loss': 1.958230551147461, 'epoch': 5.0})

In [35]:
final_model_path = "./phi2-final"
trainer.model.save_pretrained(final_model_path)

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [11]:
base_model = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto"
)

model-00001-of-00002.safetensors:  75%|#######4  | 3.73G/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [16]:
ft_model = PeftModel.from_pretrained(model,"/kaggle/input/phi-2-snli-final/transformers/default/1/phi-2-snli-final")

In [17]:
import re
from tqdm import tqdm
import torch

label_mapping = {
    0: "entailment",
    1: "neutral",
    2: "contradiction"
}

In [18]:
def extract_label_from_output(output):
    # Adjust the regex to allow optional quotation marks around the answer
    match = re.search(r'Answer:\s*"?\b(entailment|contradiction|neutral)\b"?', output, re.IGNORECASE)
    if match:
        return match.group(1).lower()  # Convert to lowercase for consistency
    # Fallback if label is unclear
    return "unknown"


def get_model_answer(model, tokenizer, premise, hypothesis, max_length=10):
    # Construct prompt more clearly to guide the model
    prompt = (
        f"Read the following premise and hypothesis carefully, and determine their relationship.\n\n"
        f"Premise: \"{premise}\"\n"
        f"Hypothesis: \"{hypothesis}\"\n\n"
        f"Choose the best answer:\n"
        f"- If the hypothesis must be true based on the premise, answer \"entailment.\"\n"
        f"- If the hypothesis could be true or false based on the premise, answer \"neutral.\"\n"
        f"- If the hypothesis is false based on the premise, answer \"contradiction.\"\n\n"
        f"Answer:"
    )

    # Tokenize input with padding and send to model's device
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)

    # Generate model output without extra tokens
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=max_length)

    # Decode response and extract clean answer
    answer = tokenizer.decode(output[0], skip_special_tokens=True).strip()
    predicted_label = extract_label_from_output(answer)
#     print(f"Model output: {answer}\nPredicted label: {predicted_label}\n")
    return predicted_label

In [19]:
def calculate_accuracy(model, dataset, tokenizer):
    model.eval()
    correct = 0
    total = 100
    i=1

    for example in tqdm(dataset):
        premise = example['premise']
        hypothesis = example['hypothesis']
        true_label_text = label_mapping[example['label']]  # Convert numerical label to text

        predicted_label = get_model_answer(model, tokenizer, premise, hypothesis)
        print(predicted_label)
        print(true_label_text)
        print()


        if predicted_label == true_label_text:
            correct += 1

        # Calculate and print running accuracy at each step
        running_accuracy = correct / i
        print(f"Step {i}/{total} - Running Accuracy: {running_accuracy:.2%}")
        i+=1

    accuracy = correct / total
    return accuracy


In [18]:
# Run evaluation on the base model
print("Evaluating base model...")
base_model_accuracy = calculate_accuracy(model, test_dataset, tokenizer)

Evaluating base model...


  1%|          | 1/100 [00:01<03:09,  1.92s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


neutral
neutral

Step 1/100 - Running Accuracy: 100.00%


  2%|▏         | 2/100 [00:03<02:39,  1.62s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


entailment
entailment

Step 2/100 - Running Accuracy: 100.00%


...

entailment
contradiction

Step 98/100 - Running Accuracy: 31.63%


 99%|█████████▉| 99/100 [02:23<00:01,  1.43s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


entailment
contradiction

Step 99/100 - Running Accuracy: 31.31%


100%|██████████| 100/100 [02:25<00:00,  1.45s/it]

entailment
neutral

Step 100/100 - Running Accuracy: 31.00%


In [20]:
print("Evaluating fine-tuned model...")
fine_tuned_model_accuracy = calculate_accuracy(ft_model, test_dataset, tokenizer)

Evaluating fine-tuned model...


  1%|          | 1/100 [00:03<05:10,  3.13s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


contradiction
neutral

Step 1/100 - Running Accuracy: 0.00%


  2%|▏         | 2/100 [00:04<03:25,  2.10s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


entailment
entailment

Step 2/100 - Running Accuracy: 50.00%


...

contradiction
contradiction

Step 98/100 - Running Accuracy: 51.02%


 99%|█████████▉| 99/100 [02:18<00:01,  1.35s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


contradiction
contradiction

Step 99/100 - Running Accuracy: 51.52%


100%|██████████| 100/100 [02:20<00:00,  1.40s/it]

unknown
neutral

Step 100/100 - Running Accuracy: 51.00%
